<a href="https://colab.research.google.com/github/sujoy1968/SBGenAIPEFT/blob/main/PEFT_SB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lightweight Fine-Tuning Project**
TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:


## Loading and Evaluating a Foundation Model
TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

## Loading Dependencies

In [ ]:
!pip install -U transformers
!pip install -q peft
!pip install -q evaluate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is inc

## Download the dataset from HuggingFace
Download "rotten_tomatoes" datset from HuggingFace. This dataset has the following splits:

1. train
2. validation
3. test




In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})

## Pre-process datasets
Convert the text to tokens in the dataset. Use lambda function to tokenize all examples.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "gpt2"
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True

def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=128)

splits = ["train", "validation",  "test"]

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        tokenize_function, batched=True
    )


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

## Download Pre-trained model from Huggingface
Download GPT-2 LLM from HuggingFace. We will be using this model for Sentiment Analysis for moview reviews from Rotten Tomatoes.

In [ ]:
from transformers import AutoModelForSequenceClassification


pt_model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

pt_model.config.pad_token_id = pt_model.config.eos_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(pt_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


## Evaluate GPT-2 using HuggingFace Trainer API

In [ ]:
import torch
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


training_args =  TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_eval=True,
    logging_dir="./logs",
)

trainer = Trainer(
    model=pt_model,
    args=training_args,
    eval_dataset = tokenized_dataset["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

print(f"Pre-trained GPT-2 evaluation results: {eval_results}")

Pre-trained GPT-2 evaluation results: {'eval_loss': 7.139991283416748, 'eval_accuracy': 0.50093808630394, 'eval_runtime': 3.3764, 'eval_samples_per_second': 315.717, 'eval_steps_per_second': 39.687}


## Performing Parameter-Efficient Fine-Tuning
TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

### Configure LORA into GPT-2 model¶

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

In [ ]:
lora_model = get_peft_model(pt_model, lora_config)

# check the number of trainable parameters

lora_model.print_trainable_parameters()

trainable params: 589,824 || all params: 125,031,168 || trainable%: 0.4717


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## Create Training Loop¶

In [ ]:
lora_training_args =  TrainingArguments(
    output_dir="./results_lora",
    learning_rate = 5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs_lora",
    logging_steps=10,
)

# LoRA takes in "labels", not "label" so we need to rename the
# training and testing sets
tokenized_dataset_lora = dataset.map(
    tokenize_function,
    batched=True,
    # remove_columns=["text"],
)

#tokenized_dataset_lora.rename_column('label', 'labels')
train_lora = tokenized_dataset_lora['train'].rename_column('label', 'labels')
eval_lora = tokenized_dataset_lora['validation'].rename_column('label', 'labels')
test_lora = tokenized_dataset_lora['test'].rename_column('label', 'labels')

train_lora

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 8530
})

In [ ]:
# Initialize the Trainer
lora_trainer = Trainer(
    model=lora_model,
    args=lora_training_args,
    train_dataset = train_lora,
    eval_dataset = eval_lora,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Train the model

lora_trainer.train()

# Save the LoRA Model
lora_model.save_pretrained("gpt2-lora")

Epoch,Training Loss,Validation Loss
1,0.613200,No log
2,0.350100,No log
3,0.427400,No log


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

## Performing Inference with a PEFT Model
TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

### Load saved LoRA model

In [ ]:
from peft import AutoPeftModelForCausalLM


lora_model = AutoPeftModelForCausalLM.from_pretrained("gpt2-lora")

lora_model.config.pad_token_id = lora_model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluate LoRA model with Rotten Tomatoes dataset¶

In [ ]:
training_args =  TrainingArguments(
    output_dir="./results_lora",
    per_device_eval_batch_size=8,
    do_eval=True,
    logging_dir="./logs_lora",
)


trainer = Trainer(
    model=lora_model,
    args=training_args,
    eval_dataset = test_lora,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate LoRA model

lora_eval_results=trainer.evaluate()

print(f"LoRA fine-tined GPT-2 evaluation results: {lora_eval_results}")

LoRA fine-tined GPT-2 evaluation results: {'eval_loss': 0.5670171976089478, 'eval_accuracy': 0.7560975609756098, 'eval_runtime': 2.4925, 'eval_samples_per_second': 427.683, 'eval_steps_per_second': 53.761}


### Compare the Evaluation resiults of LoRA fine-tuned with the original GPT2

In [ ]:
def compare_results(original_results, finetuned_results):
    print("\nComparison of Original and LoRA Fine-Tuned Model Performance:")
    for key in original_results.keys():
        if key in finetuned_results:
            print(f"{key}:")
            print(f"  Original: {original_results[key]}")
            print(f"  LoRA Fine-Tuned: {finetuned_results[key]}")

compare_results(eval_results, lora_eval_results)


Comparison of Original and LoRA Fine-Tuned Model Performance:
eval_loss:
  Original: 7.139991283416748
  LoRA Fine-Tuned: 0.5670171976089478
eval_accuracy:
  Original: 0.50093808630394
  LoRA Fine-Tuned: 0.7560975609756098
eval_runtime:
  Original: 3.3764
  LoRA Fine-Tuned: 2.4925
eval_samples_per_second:
  Original: 315.717
  LoRA Fine-Tuned: 427.683
eval_steps_per_second:
  Original: 39.687
  LoRA Fine-Tuned: 53.761


### Fine-tune using QLoRA

In [ ]:
# Install bitsandbytes package
!pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00


### Implement QLoRA Fine-tuning

**Create QLora Model**

In [ ]:
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb

# Load the pre-trained GPT-2 model with quantization
bnb_config = BitsAndBytesConfig(
    quantization_mode='qlora',
    load_in_4bit=True
)


q_model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
    config=bnb_config,
)
q_model.config.pad_token_id = q_model.config.eos_token_id

# Apply LoRA to the quantized model
qlora_model = get_peft_model(q_model, lora_config)

# check the number of trainable parameters

qlora_model.print_trainable_parameters()

Unused kwargs: ['quantization_mode']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 589,824 || all params: 125,031,168 || trainable%: 0.4717


**Create Training Loop**

In [ ]:
# Define new training arguments for fine-tuning with qLoRA


qlora_training_args =  TrainingArguments(
    output_dir="./results_qlora",
    learning_rate = 5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs_qlora",
    logging_steps=10,
)


# Initialize the Trainer for QLoRA Fine-tuning
qlora_trainer = Trainer(
    model=qlora_model,
    args=qlora_training_args,
    train_dataset = train_lora,
    eval_dataset = eval_lora,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model

qlora_trainer.train()

# Save the QLoRA Model
qlora_model.save_pretrained("gpt2-qlora")


Epoch,Training Loss,Validation Loss
1,0.552200,No log
2,0.306500,No log
3,0.402500,No log


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

### Load saved QLoRA Model

In [ ]:
qlora_model = AutoPeftModelForCausalLM.from_pretrained("gpt2-qlora")

qlora_model.config.pad_token_id = qlora_model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluate QLoRA

In [ ]:
training_args =  TrainingArguments(
    output_dir="./results_qlora",
    per_device_eval_batch_size=8,
    do_eval=True,
    logging_dir="./logs_qlora",
)


trainer = Trainer(
    model=qlora_model,
    args=training_args,
    eval_dataset = test_lora,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate QLoRA model

qlora_eval_results=trainer.evaluate()

print(f"QLoRA fine-tined GPT-2 evaluation results: {qlora_eval_results}")

QLoRA fine-tined GPT-2 evaluation results: {'eval_loss': 0.3638686537742615, 'eval_accuracy': 0.850844277673546, 'eval_runtime': 2.442, 'eval_samples_per_second': 436.521, 'eval_steps_per_second': 54.872}


### Compare the Perfomance of GPT-2, LoRA and QLoRA

In [ ]:
def compare_all_results(original_results, lora_results, qlora_results):
    print("\nComparison of Original, LoRA, and qLoRA Model Performance:")
    for key in original_results.keys():
        if key in lora_results and key in qlora_results:
            print(f"{key}:")
            print(f"  Original: {original_results[key]}")
            print(f"  LoRA: {lora_results[key]}")
            print(f"  qLoRA: {qlora_results[key]}")

compare_all_results(eval_results, lora_eval_results, qlora_eval_results)


Comparison of Original, LoRA, and qLoRA Model Performance:
eval_loss:
  Original: 7.139991283416748
  LoRA: 0.5670171976089478
  qLoRA: 0.3638686537742615
eval_accuracy:
  Original: 0.50093808630394
  LoRA: 0.7560975609756098
  qLoRA: 0.850844277673546
eval_runtime:
  Original: 3.3764
  LoRA: 2.4925
  qLoRA: 2.442
eval_samples_per_second:
  Original: 315.717
  LoRA: 427.683
  qLoRA: 436.521
eval_steps_per_second:
  Original: 39.687
  LoRA: 53.761
  qLoRA: 54.872


### Using IA3 for fine-tuning GPT-2

### Load model and IA3 Config

In [ ]:
from peft import IA3Config, TaskType

i_model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)
i_model.config.pad_token_id = i_model.config.eos_token_id

ia3_config = IA3Config(
    peft_type="IA3",
    task_type=TaskType.SEQ_CLS,

)

ia3_model = get_peft_model(i_model, ia3_config)

# check the number of trainable parameters

ia3_model.print_trainable_parameters()



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 66,048 || all params: 124,507,392 || trainable%: 0.0530


/usr/local/lib/python3.10/dist-packages/peft/tuners/ia3/model.py:131: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


### Fine-tune the Model with IA3 and save the model

In [ ]:
# Define new training arguments for fine-tuning with AdamW


ia3_training_args =  TrainingArguments(
    output_dir="./results_ia3",
    learning_rate = 5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs_ia3",
    logging_steps=10,
)


# Initialize the Trainer for QLoRA Fine-tuning
ia3_trainer = Trainer(
    model=ia3_model,
    args=ia3_training_args,
    train_dataset = train_lora,
    eval_dataset = eval_lora,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


# Train the model

ia3_trainer.train()

# Save the IA3 Model
ia3_model.save_pretrained("gpt2-ia3")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.647000,0.576455,0.751407
2,0.553200,0.520816,0.782364


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Evaluate IA3 Fine-tuned model

In [ ]:
training_args =  TrainingArguments(
    output_dir="./results_ia3",
    per_device_eval_batch_size=8,
    do_eval=True,
    logging_dir="./logs_ia3",
)


trainer = Trainer(
    model=ia3_model,
    args=training_args,
    eval_dataset = test_lora,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate IA3 model

ia3_eval_results=trainer.evaluate()

print(f"IA3 fine-tined GPT-2 evaluation results: {ia3_eval_results}")

IA3 fine-tined GPT-2 evaluation results: {'eval_loss': 0.5285229086875916, 'eval_accuracy': 0.7579737335834896, 'eval_runtime': 2.3939, 'eval_samples_per_second': 445.291, 'eval_steps_per_second': 55.975}


### Compare Performance: GPT2, LoRA, IA3

In [ ]:
def compare_all_results(original_results, lora_results, ia3_results):
    print("\nComparison of Original, LoRA, and qLoRA Model Performance:")
    for key in original_results.keys():
        if key in lora_results and key in ia3_results:
            print(f"{key}:")
            print(f"  Original: {original_results[key]}")
            print(f"  LoRA: {lora_results[key]}")
            print(f"  IA3: {ia3_results[key]}")

compare_all_results(eval_results, lora_eval_results, ia3_eval_results)


Comparison of Original, LoRA, and qLoRA Model Performance:
eval_loss:
  Original: 7.139991283416748
  LoRA: 0.5670171976089478
  IA3: 0.5285229086875916
eval_accuracy:
  Original: 0.50093808630394
  LoRA: 0.7560975609756098
  IA3: 0.7579737335834896
eval_runtime:
  Original: 3.3764
  LoRA: 2.4925
  IA3: 2.3939
eval_samples_per_second:
  Original: 315.717
  LoRA: 427.683
  IA3: 445.291
eval_steps_per_second:
  Original: 39.687
  LoRA: 53.761
  IA3: 55.975
